# VITK MiniProject
### By Chloé Magnier & Axelle Lorin

In [ ]:
import os
import sys
import vtk
import itk
import numpy as np

In [ ]:
DATAPATH = "./Data/"
CASE6_GRE1 = "case6_gre1.nrrd"
CASE6_GRE2 = "case6_gre2.nrrd"

# Step 1: Read the two nrrd files using ttk.

In [ ]:
pixel_type = itk.F
dimension = 3
image_type = itk.Image[pixel_type, dimension]

def display(*images):
    left_viewport = [0.0, 0.0, 0.33, 1.0]
    middle_viewport = [0.33, 0.0, 0.66, 1.0]
    right_viewport = [0.66, 0.0, 1.0, 1.0]
    viewports = [left_viewport, middle_viewport, right_viewport]
    render_window = vtk.vtkRenderWindow()
    for image in images:
        vtk_image = itk.vtk_image_from_image(image)
        scalar_range = vtk_image.GetScalarRange()
        data_type = vtk_image.GetScalarType()
        print(f"Scalar range: {scalar_range}")
        print(f"Data type: {data_type}")

        cmap = vtk.vtkColorTransferFunction()

        cmap.AddRGBPoint(0, 0.0, 0.0, 0.0)
        cmap.AddRGBPoint(256,0.25,0.25,0.25)
        cmap.AddRGBPoint(512,0.5,0.5,0.5)
        cmap.AddRGBPoint(768,0.75,0.75,0.75)
        cmap.AddRGBPoint(1021,1.0,1.0,1.0)
        cmap.AddRGBPoint(scalar_range[1],1.0,1.0,1.0)

        oppacity = vtk.vtkPiecewiseFunction()
        oppacity.AddPoint(0, 0.0)
        oppacity.AddPoint(100, 0.0)
        oppacity.AddPoint(1021, 1.0)
        oppacity.AddPoint(scalar_range[1], 1.0)


        mapper = vtk.vtkSmartVolumeMapper()
        mapper.SetInputData(vtk_image)
        mapper.SetBlendModeToComposite()

        volumeProperty = vtk.vtkVolumeProperty()
        volumeProperty.SetColor(cmap)
        volumeProperty.SetScalarOpacity(oppacity)
        volumeProperty.SetInterpolationTypeToLinear()
        volumeProperty.ShadeOn()

        volume = vtk.vtkVolume()
        volume.SetMapper(mapper)
        volume.SetProperty(volumeProperty)
        volume.Update()
        renderer = vtk.vtkRenderer()
        renderer.AddVolume(volume)
        renderer.SetBackground(1,1,1)
        renderer.SetViewport(viewports[images.index(image)])
        render_window.AddRenderer(renderer)


    render_window.SetSize(1200, 400)
    render_window.GetRenderers().GetFirstRenderer().GetActiveCamera().SetParallelProjection(1)
    for rend in render_window.GetRenderers():
        rend.ResetCamera()
        rend.SetActiveCamera(render_window.GetRenderers().GetFirstRenderer().GetActiveCamera())

    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetInteractorStyle(vtk.vtkInteractorStyleTerrain())
    render_window_interactor.SetRenderWindow(render_window)

    render_window.Render()
    render_window_interactor.Start()

In [ ]:
def compare(fixed_img, recaled_image):
    """
    Compare the two images using vtk
    Display the fixed image on red and the recaled image on blue
    """
    vtk_fixed = itk.vtk_image_from_image(fixed_img)
    vtk_recaled = itk.vtk_image_from_image(recaled_image)
    fixed_scalar_range = vtk_fixed.GetScalarRange()
    recaled_scalar_range = vtk_recaled.GetScalarRange()
    cmap_fixed = vtk.vtkColorTransferFunction()
    cmap_fixed.AddRGBPoint(fixed_scalar_range[0], 0.0, 0.0, 0.0)
    cmap_fixed.AddRGBPoint(100, 0.0, 0.0, 0.0)
    cmap_fixed.AddRGBPoint(fixed_scalar_range[1], 1.0, 0.0, 0.0)
    cmap_recaled = vtk.vtkColorTransferFunction()
    cmap_recaled.AddRGBPoint(recaled_scalar_range[0], 0.0, 0.0, 0.0)
    cmap_recaled.AddRGBPoint(100, 0.0, 0.0, 0.0)
    cmap_recaled.AddRGBPoint(recaled_scalar_range[1], 0.0, 0.0, 1.0)

    recaled_contour = vtk.vtkContourFilter()
    recaled_contour.SetInputData(vtk_recaled)
    recaled_contour.SetNumberOfContours(1)
    fixed_contour = vtk.vtkContourFilter()
    fixed_contour.SetNumberOfContours(1)
    fixed_contour.SetInputData(vtk_fixed)
    fixed_contour_mapper = vtk.vtkPolyDataMapper()
    recaled_contour_mapper = vtk.vtkPolyDataMapper()



    fixed_contour_mapper.SetInputConnection(fixed_contour.GetOutputPort())


    recaled_contour_mapper.SetInputConnection(recaled_contour.GetOutputPort())


    fixed_contour_actor = vtk.vtkActor()
    fixed_contour_actor.SetMapper(fixed_contour_mapper)
    recaled_contour_actor = vtk.vtkActor()
    recaled_contour_actor.SetMapper(recaled_contour_mapper)



    opacity_fixed = vtk.vtkPiecewiseFunction()
    opacity_fixed.AddPoint(fixed_scalar_range[0], 0.0)
    opacity_fixed.AddPoint(100, 0.0)
    opacity_fixed.AddPoint(fixed_scalar_range[1], 1.0)

    opacity_recaled = vtk.vtkPiecewiseFunction()
    opacity_recaled.AddPoint(recaled_scalar_range[0], 0.0)
    opacity_recaled.AddPoint(100, 0.0)
    opacity_recaled.AddPoint(recaled_scalar_range[1], 1.0)

    mapper_fixed = vtk.vtkSmartVolumeMapper()
    mapper_fixed.SetInputData(vtk_fixed)
    mapper_fixed.SetBlendModeToComposite()

    mapper_recaled = vtk.vtkSmartVolumeMapper()
    mapper_recaled.SetInputData(vtk_recaled)
    mapper_recaled.SetBlendModeToComposite()

    volumeProperty_fixed = vtk.vtkVolumeProperty()
    volumeProperty_fixed.SetColor(cmap_fixed)
    volumeProperty_fixed.SetScalarOpacity(opacity_fixed)
    volumeProperty_fixed.SetInterpolationTypeToLinear()
    volumeProperty_fixed.ShadeOff()

    volumeProperty_recaled = vtk.vtkVolumeProperty()
    volumeProperty_recaled.SetColor(cmap_recaled)
    volumeProperty_recaled.SetScalarOpacity(opacity_recaled)
    volumeProperty_recaled.SetInterpolationTypeToLinear()
    volumeProperty_recaled.ShadeOff()


    volume_fixed = vtk.vtkVolume()
    volume_fixed.SetMapper(mapper_fixed)
    volume_fixed.SetProperty(volumeProperty_fixed)

    volume_recaled = vtk.vtkVolume()
    volume_recaled.SetMapper(mapper_recaled)
    volume_recaled.SetProperty(volumeProperty_recaled)

    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume_fixed)
    renderer.AddVolume(volume_recaled)
    renderer.AddActor(fixed_contour_actor)
    renderer.AddActor(recaled_contour_actor)
    renderer.SetBackground(1,1,1)
    renderer.ResetCamera()
    renderer.GetActiveCamera().SetParallelProjection(1)
    renderer.UseDepthPeelingForVolumesOn()
    renderer.SetUseDepthPeeling(1)



    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)

    renderWindowInteractor = vtk.vtkRenderWindowInteractor()
    renderWindowInteractor.SetInteractorStyle(vtk.vtkInteractorStyleTerrain())

    renderWindowInteractor.SetRenderWindow(renderWindow)

    renderWindow.Render()
    renderWindowInteractor.Start()

In [ ]:
fixed_image = itk.imread(DATAPATH+CASE6_GRE1, pixel_type)
moving_image = itk.imread(DATAPATH+CASE6_GRE2, pixel_type)

TransformType = itk.TranslationTransform[itk.D, dimension]

In [ ]:
optimizerType = itk.RegularStepGradientDescentOptimizerv4[itk.D]
MetricType = itk.MeanSquaresImageToImageMetricv4[image_type, image_type]
InterpolatorType = itk.LinearInterpolateImageFunction[image_type, itk.D]
RegistrationType = itk.ImageRegistrationMethodv4[image_type, image_type]
ResamplerFilterType = itk.ResampleImageFilter[image_type, image_type]

FixedImageType = image_type
MovingImageType = image_type

fixedInterpolator = InterpolatorType.New()
movingInterpolator = InterpolatorType.New()
metric = MetricType.New()
metric.SetFixedInterpolator(fixedInterpolator)
metric.SetMovingInterpolator(movingInterpolator)

initialTransform = TransformType.New()
optimizer = optimizerType.New()
registration = RegistrationType.New(FixedImage=fixed_image, MovingImage=moving_image, Metric=metric,
                                                     Optimizer=optimizer, InitialTransform=initialTransform)

movingInitialTransform = TransformType.New()
movingInitialParameters = movingInitialTransform.GetParameters()
movingInitialParameters[0] = 0
movingInitialParameters[1] = 0
movingInitialParameters[2] = 0
movingInitialTransform.SetParameters(movingInitialParameters)

identityTransform = TransformType.New()
identityTransform.SetIdentity()
registration.SetFixedInitialTransform(identityTransform)

registration.Update()
finalTransform = registration.GetTransform()
finalParameters = finalTransform.GetParameters()

In [ ]:
#finalTransform = TransformType.New()
#finalParameters = finalTransform.GetParameters()
#finalParameters[0] = -0.83424233797153
#finalParameters[1] = -3.541046559177671
#finalParameters[2] = -59.45504069565786
#finalTransform.SetParameters(finalParameters)

print(f"Final parameters: \n X: {finalParameters.GetElement(0)} \n Y: {finalParameters.GetElement(1)} \n Z: {finalParameters.GetElement(2)}")

resampler = itk.ResampleImageFilter.New(Input=moving_image, Transform=finalTransform, UseReferenceImage=True,
                                            ReferenceImage=fixed_image)


resampler.SetDefaultPixelValue(0.0)

resampler.Update()

compare(fixed_image, resampler.GetOutput())